# Création de nouveaux sous-titres segmentés en phrase

Ce notebook permet de créer de nouveaux fichiers de sous-titres à l'aide de deux fichiers : 
1. Un fichier contenant les phrases (sous titres)
2. Un fichier de sous-titres, sous segmenté pour que chaque sous-titre correspondent à une phrase ou à un morceau de phrase (et non deux phrases dans un même sous-titre)

In [1]:
import os
import re
import module_traitement as m 

In [162]:
# Pour Matignon - LSF 
pre_seg_file = "../data/new_segmentation_cr/"
sentence_file = "../data/sentence_matignon/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "../data/new_cr_sub/"

In [70]:
# Pour Mediapi : 
pre_seg_file = "../data/new_segmentation_mediapi/"
sentence_file = "../data/sentence_mediapi/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "../data/new_mediapi_sub/"

In [2]:
# Pour Matignon - LSF 
pre_seg_file = "test_new_seg_cr/"
sentence_file = "test_clean_sent/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "test_sent_seg/"

In [7]:
import module_traitement as m 

In [8]:
# Création de nouveaux fichiers de sous-titres
for sub,sub_name,phr,phr_name in zip(resultat,resultat_output,path_sentence,sentences_only):
    print(f"TRAITEMENT : {sub_name} && {phr_name}")
    print(f"TRAITEMENT : {sub} && {phr}")
    dict_sub = m.get_dict_vtt(sub)
    sentences = m.get_sentences(phr)
    keys = list(dict_sub.keys())
    i = 0
    j = 0
    mm = 0
    new_dict = {}
    content = ""
    while i < len(keys) and j < len(sentences):
        #print(mm)
        key = keys[i]
        value = dict_sub[key]
        unite = value['text'].strip()
        sent = sentences[j].strip()
        #print(f"'{unite} --> '{sent}'")
        # Je regarde si l'unité est dans la phrase. Si elle est dans la phrase, j'ajoute l'information start, et tant que les unités sont dans la même phrase je concatène pour ajouter l'information du texte
        if unite in sent:
            #print(f"'{unite} --> '{sent}'")
            if mm not in new_dict:
                new_dict[mm] = {"start": value['start']}
            if 'text' not in new_dict[mm]:
                new_dict[mm]["text"] = unite.strip()
            else:
                new_dict[mm]['text'] += f" {unite.strip()}"
            i = i +1

            if i == len(keys):
                new_dict[mm]['end'] = value['end']
        else:
            old_key = keys[i - 1]
            if mm in new_dict and 'end' not in new_dict[mm]:
                new_dict[mm]['end'] = dict_sub[old_key]['end']
                #print(new_dict)
                mm += 1
            j = j +1
    m.create_vtt_file(new_dict,f"{output_folder}/{sub_name}")

TRAITEMENT : -LhfYZ1ihpI.vtt && -LhfYZ1ihpI.vtt
TRAITEMENT : test_new_seg_cr/-LhfYZ1ihpI.vtt && test_clean_sent/-LhfYZ1ihpI.vtt
TRAITEMENT : -sgE2QHsskA.vtt && -sgE2QHsskA.vtt
TRAITEMENT : test_new_seg_cr/-sgE2QHsskA.vtt && test_clean_sent/-sgE2QHsskA.vtt
TRAITEMENT : 0-RtcGGzUoE.vtt && 0-RtcGGzUoE.vtt
TRAITEMENT : test_new_seg_cr/0-RtcGGzUoE.vtt && test_clean_sent/0-RtcGGzUoE.vtt
TRAITEMENT : 0IViLqFmgIY.vtt && 0IViLqFmgIY.vtt
TRAITEMENT : test_new_seg_cr/0IViLqFmgIY.vtt && test_clean_sent/0IViLqFmgIY.vtt
TRAITEMENT : 0hXvxmgHk_c.vtt && 0hXvxmgHk_c.vtt
TRAITEMENT : test_new_seg_cr/0hXvxmgHk_c.vtt && test_clean_sent/0hXvxmgHk_c.vtt
TRAITEMENT : 1AjRdJ5d_Ww.vtt && 1AjRdJ5d_Ww.vtt
TRAITEMENT : test_new_seg_cr/1AjRdJ5d_Ww.vtt && test_clean_sent/1AjRdJ5d_Ww.vtt
TRAITEMENT : 1ILfD_BjLNk.vtt && 1ILfD_BjLNk.vtt
TRAITEMENT : test_new_seg_cr/1ILfD_BjLNk.vtt && test_clean_sent/1ILfD_BjLNk.vtt
TRAITEMENT : 1MHphyCtLLE.vtt && 1MHphyCtLLE.vtt
TRAITEMENT : test_new_seg_cr/1MHphyCtLLE.vtt && test_cle

## Vérifications manuelles

On compare le fichier de phrases avec les nouveaux sous-titres, pour vérifier si on a la même chose. 

S'il y a une différence, on note le fichier et l'endroit, ce qui permet de faire une correction.

On corriger : 
- soit le fichier de phrases (si la phrases a été mal découpé par le modèle de langue)
- soit le fichier sur-segmenté, s'il y a eu un problème de segmentation

In [22]:
path_sentence = path_sentence # vérifier qu'on charge le bon fichier !
output_folder = m.lister_fichiers_with_path("test_sent_seg")
print(output_folder)

['test_sent_seg/-LhfYZ1ihpI.vtt', 'test_sent_seg/-sgE2QHsskA.vtt', 'test_sent_seg/0-RtcGGzUoE.vtt', 'test_sent_seg/0IViLqFmgIY.vtt', 'test_sent_seg/0hXvxmgHk_c.vtt', 'test_sent_seg/1AjRdJ5d_Ww.vtt', 'test_sent_seg/1ILfD_BjLNk.vtt', 'test_sent_seg/1MHphyCtLLE.vtt', 'test_sent_seg/3TEX9ruhaXo.vtt', 'test_sent_seg/6fsIXStr6w4.vtt', 'test_sent_seg/8IqhpOiPMxY.vtt', 'test_sent_seg/8ZUIw7jcaZE.vtt', 'test_sent_seg/B62_uSapEhI.vtt', 'test_sent_seg/CDsP8gaVGbg.vtt', 'test_sent_seg/CxgdUjywiDE.vtt', 'test_sent_seg/F5-w4cvC_L0.vtt', 'test_sent_seg/FDNmSH37IEk.vtt', 'test_sent_seg/G3Tz-srNvjs.vtt', 'test_sent_seg/H0dUQaWH3UE.vtt', 'test_sent_seg/H448NJiwMRI.vtt', 'test_sent_seg/K7UFz1gAzAQ.vtt', 'test_sent_seg/K7WqKupeGVk.vtt', 'test_sent_seg/KhxgwQDJpCg.vtt', 'test_sent_seg/LicjiPStTmU.vtt', 'test_sent_seg/M5SVtrVMQm0.vtt', 'test_sent_seg/NiQIWYxm8EQ.vtt', 'test_sent_seg/QXVJZkmva3s.vtt', 'test_sent_seg/RenNdZa3-QY.vtt', 'test_sent_seg/SdneZpqEo-8.vtt', 'test_sent_seg/Sy7uU8DAsfs.vtt', 'test_sen

In [23]:
def get_dict_vtt(input):
    with open(input,'r',encoding="utf-8") as f:
        lines = f.readlines()

    dict_sub = {}
    i = 0
    j = 0  

    while j < len(lines): 
        element = lines[j]
        if element.startswith("00:") or element.startswith("01:") or element.startswith("02:"):
            # Extraire le temps de début et de fin
            timing_line = element.strip().split(' --> ')
            start_time, end_time = timing_line

            text = ""
            while j + 1 < len(lines) and not lines[j + 1].startswith("00:") and not lines[j+1].startswith("01:") and not lines[j+1].startswith("02:"):
                j += 1
                content = lines[j]
                text = text + " " + content.strip()

            dict_sub[i] = {'start': start_time, 'end': end_time, 'text': text.strip()}
            i += 1

        j += 1


In [24]:
nb_file = 0
name_file = []
for sub_file, sent_file in zip(output_folder,path_sentence):
    print(sub_file,sent_file)
    sub_dict = m.get_dict_vtt(sub_file)
    sentence = m.get_sentences(sent_file)
    new_sent = []
    for k,v in sub_dict.items():
        for kk,vv in v.items():
            if kk == "text":
                new_sent.append(vv)
    differences = m.comparer_listes(new_sent, sentence)
    if differences:
        print(f"TRAITEMENT : {sub_file} && {sent_file}")
        for position, element_a, element_b in differences:
            nb_file = nb_file +1
            print(f"Différence à la position {position}: {element_a} vs {element_b}")
            name_file.append(sub_file)
            break
    else:
        continue

print(f"Nombres de fichiers contenant une erreur : {nb_file}")

test_sent_seg/-LhfYZ1ihpI.vtt test_clean_sent/-LhfYZ1ihpI.vtt
TRAITEMENT : test_sent_seg/-LhfYZ1ihpI.vtt && test_clean_sent/-LhfYZ1ihpI.vtt
Différence à la position 198: Là, la capacité de développer de l'apprentissage par podcast, etc. , pour faire de l'innovation concrète. vs Là, la capacité de développer de l'apprentissage par podcast, etc., pour faire de l'innovation concrète.
test_sent_seg/-sgE2QHsskA.vtt test_clean_sent/-sgE2QHsskA.vtt
TRAITEMENT : test_sent_seg/-sgE2QHsskA.vtt && test_clean_sent/-sgE2QHsskA.vtt
Différence à la position 177: None vs La page des petites tensions est derrière nous?
test_sent_seg/0-RtcGGzUoE.vtt test_clean_sent/0-RtcGGzUoE.vtt
TRAITEMENT : test_sent_seg/0-RtcGGzUoE.vtt && test_clean_sent/0-RtcGGzUoE.vtt
Différence à la position 146: Pourquoi et est-ce que vous allez revenir sur cette décision de ne pas obliger les soignants des départements d'outre-mer Inaudible. ? vs Pourquoi et est-ce que vous allez revenir sur cette décision de ne pas obliger les